# Use Closed-Form Policy to Play Taxi-v3

In [1]:
import sys
import logging
import imp
import itertools

import numpy as np
np.random.seed(0)
import gym

imp.reload(logging)
logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('Taxi-v3')
env.seed(0)
for key in vars(env):
    logging.info('%s: %s', key, vars(env)[key])
for key in vars(env.spec):
    logging.info('%s: %s', key, vars(env.spec)[key])

00:00:00 [INFO] env: <TaxiEnv<Taxi-v3>>
00:00:00 [INFO] action_space: Discrete(6)
00:00:00 [INFO] observation_space: Discrete(500)
00:00:00 [INFO] reward_range: (-inf, inf)
00:00:00 [INFO] metadata: {'render.modes': ['human', 'ansi']}
00:00:00 [INFO] _max_episode_steps: 200
00:00:00 [INFO] _elapsed_steps: None
00:00:00 [INFO] id: Taxi-v3
00:00:00 [INFO] entry_point: gym.envs.toy_text:TaxiEnv
00:00:00 [INFO] reward_threshold: 8
00:00:00 [INFO] nondeterministic: False
00:00:00 [INFO] max_episode_steps: 200
00:00:00 [INFO] _kwargs: {}
00:00:00 [INFO] _env_name: Taxi


In [3]:
class ClosedFormAgent:
    def __init__(self, env):
        state_n, action_n = env.observation_space.n, env.action_space.n
        v = np.zeros((env.spec.max_episode_steps+1, state_n))
        q = np.zeros((env.spec.max_episode_steps+1, state_n, action_n))
        pi = np.zeros((env.spec.max_episode_steps+1, state_n))
        for t in range(env.spec.max_episode_steps-1, -1, -1):
            for s in range(state_n):
                for a in range(action_n):
                    for p, next_s, r, d in env.P[s][a]:
                        q[t, s, a] += p * (r + (1. - float(d)) * v[t+1, next_s])
                v[t, s] = q[t, s].max()
                pi[t, s] = q[t, s].argmax()
        self.pi = pi

    def reset(self, mode=None):
        self.t = 0

    def step(self, observation, _reward, _done):
        action = self.pi[self.t, observation]
        self.t += 1
        return action

    def close(self):
        pass


agent = ClosedFormAgent(env)

In [4]:
def play_episode(env, agent, max_episode_steps=None, mode=None, render=False):
    observation, reward, done = env.reset(), 0., False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, done)
        if render:
            env.render()
        if done:
            break
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
        if max_episode_steps and elapsed_steps >= max_episode_steps:
            break
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.debug('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

00:00:02 [INFO] ==== test ====
00:00:02 [DEBUG] test episode 0: reward = 4.00, steps = 17
00:00:02 [DEBUG] test episode 1: reward = 10.00, steps = 11
00:00:02 [DEBUG] test episode 2: reward = 10.00, steps = 11
00:00:02 [DEBUG] test episode 3: reward = 11.00, steps = 10
00:00:02 [DEBUG] test episode 4: reward = 8.00, steps = 13
00:00:02 [DEBUG] test episode 5: reward = 9.00, steps = 12
00:00:02 [DEBUG] test episode 6: reward = 11.00, steps = 10
00:00:02 [DEBUG] test episode 7: reward = 6.00, steps = 15
00:00:02 [DEBUG] test episode 8: reward = 9.00, steps = 12
00:00:02 [DEBUG] test episode 9: reward = 13.00, steps = 8
00:00:02 [DEBUG] test episode 10: reward = 11.00, steps = 10
00:00:02 [DEBUG] test episode 11: reward = 9.00, steps = 12
00:00:02 [DEBUG] test episode 12: reward = 4.00, steps = 17
00:00:02 [DEBUG] test episode 13: reward = 8.00, steps = 13
00:00:02 [DEBUG] test episode 14: reward = 13.00, steps = 8
00:00:02 [DEBUG] test episode 15: reward = 4.00, steps = 17
00:00:02 [DEBU

In [5]:
env.close()